In [1]:
import sys
import os
import pandas as pd
import warnings
import json
from pydantic import BaseModel

# Add the src folder to the path
sys.path.append(os.path.abspath('../..'))
warnings.filterwarnings("ignore", category=FutureWarning)

# Import the classes
from src.data_tool.data_providers.ploygon import polygon
from src.database_tool.connect_db import ConnectDB
from src.data_tool.data_models import CompanyNews, CompanyNewsResponse

from src.llm.api_call import call_llm
from langchain_core.prompts import ChatPromptTemplate

db =ConnectDB()
engine = db.get_engine()

DB_USER and DB_PASSWORD are set


In [2]:
#fd = FinancialDatasets()
pl = polygon()

POLYGON_API_KEY is set
DB_USER and DB_PASSWORD are set


In [3]:
ticker = 'NVDA'
start_date = '2025-03-15'
end_date = '2025-04-02'

In [4]:
def prepare_news_for_llm(news_list):
    """
    Prepare company news data for an LLM by removing URLs and converting to a structured format.
    
    Args:
        news_list: List of CompanyNews objects
        
    Returns:
        List of dictionaries containing cleaned news data
    """
    processed_news = []
    
    for news_item in news_list:
        # Create a dictionary with all fields except URL
        news_dict = {
            "tickers": news_item.tickers,
            "title": news_item.title,
            "description": news_item.description,
            "published_utc": news_item.published_utc,
        }
        
        processed_news.append(news_dict)
    
    return processed_news

In [5]:
nvdia_news, nvdia_news_df = pl.get_news(ticker, start_date, end_date, limit=1000, strict=False)

Skipped 146 duplicate news records for NVDA.
All 146 news records for NVDA already exist in the database.


In [6]:
nvdia_news_df.sort_index(ascending=False)

,article_url,author,description,polygon_id,insights,keywords,published_utc,tickers,title,publisher,ticker
145,https://www.benzinga.com/government/regulation...,Piero Cingari,President Trump's new tariffs triggered a shar...,9f9c58226349b93485c4f8a796b03d103d74f74336db64...,"[{""ticker"": ""AAPL"", ""sentiment"": ""negative"", ""...","[""tariffs"", ""trade war"", ""tech stocks"", ""marke...",2025-04-02 21:35:44,"[""AAPL"", ""AMZN"", ""NVDA"", ""XLK"", ""XLY""]",Markets Reel As Trump's Tariffs Hit — Magnific...,Benzinga,NVDA
144,https://www.investing.com/analysis/liberation-...,Zain Vawda,The article discusses the potential impact of ...,44aa6413805e29973e0509ad35a8538a0bcd545f9221d6...,"[{""ticker"": ""AAPL"", ""sentiment"": ""neutral"", ""s...","[""tariffs"", ""trade war"", ""stock market"", ""S&P ...",2025-04-02 19:24:00,"[""AAPL"", ""MSFT"", ""NVDA""]",Liberation Day Tariffs: Potential Impact on th...,Investing.com,NVDA
143,https://www.fool.com/investing/2025/04/02/why-...,Jeremy Bowman,Nvidia's stock fell 13% in March due to macroe...,cfa5d6f1390722d6889932741d57a0ad27f5b512003cd8...,"[{""ticker"": ""NVDA"", ""sentiment"": ""neutral"", ""s...","[""Nvidia"", ""AI"", ""semiconductor"", ""macroeconom...",2025-04-02 18:34:50,"[""NVDA""]",Why Nvidia Stock Lost 13% in March,The Motley Fool,NVDA
142,https://www.fool.com/investing/2025/04/02/some...,Anders Bylund,Quantum computing stocks had a volatile month ...,5144eaef31e6469a48882a4c387c51e3d0b49f6ff107ce...,"[{""ticker"": ""QBTS"", ""sentiment"": ""positive"", ""...","[""quantum computing"", ""stock performance"", ""D-...",2025-04-02 16:52:36,"[""QBTS"", ""QBTS.WS"", ""QUBT"", ""RGTI"", ""RGTIW"", ""...",Why Some Quantum Computing Stocks Soared Last ...,The Motley Fool,NVDA
141,https://www.fool.com/investing/2025/04/02/core...,Jeremy Bowman,"CoreWeave, a cloud infrastructure platform pur...",2a992adbd8a6dd72e83aa9bf28b5176da249a716fa2303...,"[{""ticker"": ""CRWV"", ""sentiment"": ""neutral"", ""s...","[""CoreWeave"", ""AI"", ""IPO"", ""Nvidia"", ""Microsof...",2025-04-02 15:48:46,"[""CRWV"", ""NVDA"", ""MSFT""]",CoreWeave IPO: Is this Fast-Growing AI Stock a...,The Motley Fool,NVDA
...,...,...,...,...,...,...,...,...,...,...,...
4,https://www.globenewswire.com/news-release/202...,"Faruqi & Faruqi, Llp",Monolithic Power Systems is facing a class act...,047160c92a8411a7fff790bc60c24fb945df070254fac1...,"[{""ticker"": ""MPWR"", ""sentiment"": ""negative"", ""...","[""Monolithic Power Systems"", ""Nvidia"", ""class ...",2025-03-15 11:30:00,"[""MPWR"", ""NVDA""]",INVESTOR DEADLINE APPROACHING: Faruqi & Faruqi...,GlobeNewswire Inc.,NVDA
3,https://www.fool.com/investing/2025/03/15/as-a...,The Motley Fool,"The article discusses two AI stocks, Nvidia an...",7c30fbe666bd02d32c89088d0c1ac97c0de9c9e176b934...,"[{""ticker"": ""NVDA"", ""sentiment"": ""positive"", ""...","[""Artificial Intelligence"", ""Nvidia"", ""Meta Pl...",2025-03-15 10:45:00,"[""NVDA"", ""META""]","As Artificial Intelligence (AI) Stocks Fall, I...",The Motley Fool,NVDA
2,https://www.fool.com/investing/2025/03/15/why-...,Jose Najarro,The article discusses recent updates impacting...,59ca0d312db7266886710a26b2a491e639bf8dc9e74070...,"[{""ticker"": ""NVDA"", ""sentiment"": ""positive"", ""...","[""Nvidia"", ""AI market""]",2025-03-15 09:15:00,"[""NVDA""]",Why Nvidia Stock Investors Should Hope AI Gets...,The Motley Fool,NVDA
1,https://www.fool.com/investing/2025/03/15/arti...,The Motley Fool,Nvidia is a leading provider of GPUs for AI ap...,5214c6c7b01139fdb26cff13dae7b1c6c8c219e0fd26c1...,"[{""ticker"": ""NVDA"", ""sentiment"": ""positive"", ""...","[""Artificial Intelligence"", ""Nvidia"", ""GPU"", ""...",2025-03-15 09:15:00,"[""NVDA"", ""AMD""]",1 Artificial Intelligence (AI) Stock Millennia...,The Motley Fool,NVDA


In [7]:
LLM_news = prepare_news_for_llm(nvdia_news.results)

In [8]:
template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are a financial analyst. You are given a list of news articles and a ticker. You need to analyze the news and provide a summary of the news and the sentiment of the news.
                    You should also identify if the news mentioned any upcoming key events that could impact the stock price.
                """,
            ),
            (
                "human",
                """Based on the following news, create the investment signal:

                News Data for {ticker}:
                {llm_news}
                """,
            ),
        ]
    )

prompt = template.invoke({"llm_news": json.dumps(LLM_news, indent=2), "ticker": ticker})

In [9]:
print(prompt)

messages=[SystemMessage(content='You are a financial analyst. You are given a list of news articles and a ticker. You need to analyze the news and provide a summary of the news and the sentiment of the news.\n                    You should also identify if the news mentioned any upcoming key events that could impact the stock price.\n                ', additional_kwargs={}, response_metadata={}), HumanMessage(content='Based on the following news, create the investment signal:\n\n                News Data for NVDA:\n                [\n  {\n    "tickers": [\n      "NVDA",\n      "AMZN",\n      "MSFT"\n    ],\n    "title": "Prediction: This Will Be Nvidia\'s Next Big Move (and It Will Start on March 18)",\n    "description": "Nvidia\'s long-term growth prospects remain strong despite recent market slump. The company is expected to focus more on innovation, with potential for faster chip updates and a focus on reasoning inference, which could drive high-level growth over time.",\n    "publ

In [10]:
class short_term_sentiment(BaseModel):
    short_term_sentiment: str
    short_term_confidence: float
    short_term_reasoning: str
    
class long_term_sentiment(BaseModel):
    long_term_sentiment: str
    long_term_confidence: float
    long_term_reasoning: str
    
class key_event_outlook(BaseModel):
    event: str
    event_date: str
    event_type: str
    event_impact: str
    event_reasoning: str
    
class sentiment(BaseModel):
    short_term_sentiment: short_term_sentiment
    long_term_sentiment: long_term_sentiment
    key_events_outlook: list[key_event_outlook]
    


In [11]:
model_name = "gemini-2.0-flash"
model_provider = "Gemini"
pydantic_model = sentiment
max_retries = 3

In [12]:
sentiment_result = call_llm(prompt, model_name, model_provider, pydantic_model, max_retries=max_retries)
print(sentiment_result)

short_term_sentiment=short_term_sentiment(short_term_sentiment='Neutral', short_term_confidence=0.6, short_term_reasoning="Nvidia's stock has dropped 12% year-to-date and 20% from its January highs, despite the company reporting strong Q4 and full-year earnings. The decline is attributed to concerns over trade tariffs and export restrictions, which could impact semiconductor chip sales.") long_term_sentiment=long_term_sentiment(long_term_sentiment='Positive', long_term_confidence=0.8, long_term_reasoning="Nvidia is a leader in the growing generative AI market, with its products becoming increasingly important as the technology takes hold across industries. Despite the stock's recent surge, the article argues there is still time to invest in Nvidia due to the vast potential of the AI opportunity.") key_events_outlook=[key_event_outlook(event='GPU Technology Conference', event_date='2025-03-18', event_type='Conference', event_impact='Positive', event_reasoning='Expected to unveil next-ge

In [13]:
sentiment_result

sentiment(short_term_sentiment=short_term_sentiment(short_term_sentiment='Neutral', short_term_confidence=0.6, short_term_reasoning="Nvidia's stock has dropped 12% year-to-date and 20% from its January highs, despite the company reporting strong Q4 and full-year earnings. The decline is attributed to concerns over trade tariffs and export restrictions, which could impact semiconductor chip sales."), long_term_sentiment=long_term_sentiment(long_term_sentiment='Positive', long_term_confidence=0.8, long_term_reasoning="Nvidia is a leader in the growing generative AI market, with its products becoming increasingly important as the technology takes hold across industries. Despite the stock's recent surge, the article argues there is still time to invest in Nvidia due to the vast potential of the AI opportunity."), key_events_outlook=[key_event_outlook(event='GPU Technology Conference', event_date='2025-03-18', event_type='Conference', event_impact='Positive', event_reasoning='Expected to un

In [14]:
print(type(sentiment_result))

<class '__main__.sentiment'>
